In [1]:
import pandas as pd
from obspy.geodetics import locations2degrees, degrees2kilometers
from tqdm import tqdm

In [2]:
events_nwa_shelf_trench  = pd.read_csv('../data/datasets_nwa_shelf_trench/new_events.csv',index_col=0)
events_nwa_shore  = pd.read_csv('../data/datasets_nwa_shore/new_events.csv',index_col=0)
events_or_shelf_trench  = pd.read_csv('../data/datasets_or_shelf_trench/new_events.csv',index_col=0)
events_or_shore  = pd.read_csv('../data/datasets_or_shore/new_events.csv',index_col=0)
events_pnsn_jdf  = pd.read_csv('../data/datasets_pnsn_jdf/new_events.csv',index_col=0)
events_pnsn_nor = pd.read_csv('../data/datasets_pnsn_nor/new_events.csv',index_col=0)
events_pnsn_sor = pd.read_csv('../data/datasets_pnsn_sor/new_events.csv',index_col=0)
events_pnsn_wa = pd.read_csv('../data/datasets_pnsn_wa/new_events.csv',index_col=0)
events_swa_shelf_trench  = pd.read_csv('../data/datasets_swa_shelf_trench/new_events.csv',index_col=0)
events_swa_shore  = pd.read_csv('../data/datasets_swa_shore/new_events.csv',index_col=0)


In [3]:
events_nwa_shelf_trench['datetime'] = pd.to_datetime(events_nwa_shelf_trench['datetime'], utc = True)
events_nwa_shore['datetime'] = pd.to_datetime(events_nwa_shore['datetime'], utc = True)
events_or_shelf_trench['datetime'] = pd.to_datetime(events_or_shelf_trench['datetime'], utc = True)
events_or_shore['datetime'] = pd.to_datetime(events_or_shore['datetime'], utc = True)
events_pnsn_jdf['datetime'] = pd.to_datetime(events_pnsn_jdf['datetime'], utc = True)
events_pnsn_nor['datetime'] = pd.to_datetime(events_pnsn_nor['datetime'], utc = True)
events_pnsn_sor['datetime'] = pd.to_datetime(events_pnsn_sor['datetime'], utc = True)
events_pnsn_wa['datetime'] = pd.to_datetime(events_pnsn_wa['datetime'], utc = True)
events_swa_shelf_trench['datetime'] = pd.to_datetime(events_swa_shelf_trench['datetime'], utc = True)
events_swa_shore['datetime'] = pd.to_datetime(events_swa_shore['datetime'], utc = True)



In [2]:
import pandas as pd
from obspy.geodetics import locations2degrees, degrees2kilometers

# Load all event data into a list of DataFrames
event_files = [
    '../data/datasets_nwa_shelf_trench/new_events.csv',
    '../data/datasets_nwa_shore/new_events.csv',
    '../data/datasets_or_shelf_trench/new_events.csv',
    '../data/datasets_or_shore/new_events.csv',
    '../data/datasets_pnsn_jdf/new_events.csv',
    '../data/datasets_pnsn_nor/new_events.csv',
    '../data/datasets_pnsn_sor/new_events.csv',
    '../data/datasets_pnsn_wa/new_events.csv',
    '../data/datasets_swa_shelf_trench/new_events.csv',
    '../data/datasets_swa_shore/new_events.csv'
]

# Read all event files into DataFrames
dfs = [pd.read_csv(file, index_col=0) for file in event_files]

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs)

# Sort the DataFrame by 'datetime'
merged_df['datetime'] = pd.to_datetime(merged_df['datetime'])
merged_df = merged_df.sort_values(by='datetime').reset_index(drop=True)

# Define thresholds
time_threshold = pd.Timedelta(seconds=10)
dist_threshold = 25  # in kilometers

# Function to calculate distance between two events
def calculate_distance(lat1, lon1, lat2, lon2):
    degrees = locations2degrees(lat1, lon1, lat2, lon2)
    return degrees2kilometers(degrees)

# Iterate through the DataFrame and remove events within the thresholds
filtered_events = []
for i, event in merged_df.iterrows():
    if not filtered_events:
        filtered_events.append(event)
        continue
    
    last_event = filtered_events[-1]
    # print(event['idx'],last_event['idx'])
    time_diff = event['datetime'] - last_event['datetime']
    distance = calculate_distance(event['latitude'], event['longitude'], last_event['latitude'], last_event['longitude'])
    
    if time_diff > time_threshold or distance > dist_threshold:
        filtered_events.append(event)

# Convert the filtered events back to a DataFrame
filtered_df = pd.DataFrame(filtered_events)

# Save the merged and filtered catalog to a new CSV file
filtered_df.to_csv('../data/datasets_all_regions/merged_new_events1.csv', index=False)

In [3]:
filtered_df

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
0,0,0,2011-01-01 04:58:03.504768+00:00,219.223650,447.470689,0.390625,6,48.487194,-122.534153,0.390625,0,1940523,0.030308,D03D,S,1.293858e+09,2011-01-01 04:58:03.504768+00:00
1,13,3,2011-01-01 19:52:46.661153+00:00,140.013069,351.633757,8.203125,6,47.648376,-123.636385,8.203125,3,1049251,1.326894,OFR,P,1.293912e+09,2011-01-01 19:52:46.661153+00:00
2,19,4,2011-01-01 21:15:56.139179+00:00,181.291823,464.635512,19.140625,10,48.653486,-123.039309,19.140625,4,1895188,0.487547,LRIV,S,1.293917e+09,2011-01-01 21:15:56.139179+00:00
3,29,5,2011-01-02 02:10:45.376984+00:00,-0.557821,518.990787,49.609375,17,49.168544,-125.507649,49.609375,5,1121863,0.554185,BTB,P,1.293934e+09,2011-01-02 02:10:45.376984+00:00
5,46,6,2011-01-02 04:23:38.382767+00:00,234.842638,457.483502,0.390625,7,48.571463,-122.317539,0.390625,6,1961027,-0.158275,LZB,S,1.293942e+09,2011-01-02 04:23:38.382767+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41625,68797,8958,2015-12-30 10:12:04.084015+00:00,224.801860,458.913904,9.765625,8,48.587986,-122.452629,9.765625,8958,1475194,-0.269381,CLRS,S,1.451470e+09,2015-12-30 10:12:04.084015+00:00
41626,68805,8959,2015-12-30 14:04:12.055910+00:00,126.625365,540.446816,5.859375,6,49.348330,-123.757253,5.859375,8959,374144,-0.009508,CLRS,P,1.451484e+09,2015-12-30 14:04:12.055910+00:00
41627,13816,2192,2015-12-30 17:21:56.886771+00:00,76.421476,-173.323761,19.140625,7,42.936197,-124.563756,19.140625,2192,182969,-0.059408,DBO,P,1.451496e+09,2015-12-30 17:21:56.886771+00:00
41628,68811,8961,2015-12-31 01:10:15.705291+00:00,96.503032,458.913904,38.671875,18,48.620884,-124.191036,38.671875,8961,595258,-0.121397,CLRS,P,1.451524e+09,2015-12-31 01:10:15.705291+00:00


In [4]:
ls_df = [events_nwa_shelf_trench,events_nwa_shore,events_or_shelf_trench,
         events_or_shore,events_pnsn_jdf,events_pnsn_nor,events_pnsn_sor,
         events_pnsn_wa,events_swa_shelf_trench,events_swa_shore]


In [ ]:
# time_threshold = 10 # in seconds
# dist_threshold =25 # in kilometers

# for i in range(len(ls_df)):
#     df1 = ls_df[i]
#     for j in range(len(df1)):
#         event = j
#         for k in range(len(ls_df)):
#             df2 = ls_df[k]
#             t11 = df1.loc[j]['datetime']
#             olat = df1.loc[j]['latitude']
#             olon = df1.loc[j]['longitude']
            
#             condition = ((df2['datetime'] <= t11 - pd.Timedelta(seconds=time_threshold)) & \
#                         (df2['datetime'] >= t11 + pd.Timedelta(seconds=time_threshold)) & \
#                         (degrees2kilometers(locations2degrees(olat, olon, df2['latitude'], df2['longitude'])) >= dist_threshold)) | \
#                         (df2.index == j)
                        
#             ls_df[k] = df2.loc[condition]

# df_merged = pd.concat(ls_df, axis =0, ignore_index=True)
# df_merged

In [ ]:
# time_threshold = 10 # in seconds
# dist_threshold =25 # in kilometers

# # Drop the events in df2 that matches each event in df1. This would drop one of the two events in different catalogs 
# # that fall within the time and distance threshold.
# for i in range(len(ls_df)-1):
#     df1 = ls_df[i]
#     for j in range(len(df1)):
#         event = j
#         for k in range(len(ls_df)-1):
#             df2 = ls_df[k+1]
#             t11 = df1.loc[j]['datetime']
#             olat = df1.loc[j]['latitude']
#             olon = df1.loc[j]['longitude']
            
#             condition = (df2['datetime'] <= t11 - pd.Timedelta(seconds=time_threshold)) & \
#                      (df2['datetime'] >= t11 + pd.Timedelta(seconds=time_threshold)) & \
#                      (degrees2kilometers(locations2degrees(olat, olon, df2['latitude'], df2['longitude'])) >= dist_threshold)                        
#             ls_df[k+1]=df2.loc[condition]

# df_merged = pd.concat(ls_df, axis =0, ignore_index=True)
# df_merged


In [ ]:
ls_df

In [2]:
filtered_df

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
0,0,0,2011-01-01 04:58:03.504768+00:00,219.223650,447.470689,0.390625,6,48.487194,-122.534153,0.390625,0,1940523,0.030308,D03D,S,1.293858e+09,2011-01-01 04:58:03.504768+00:00
1,13,3,2011-01-01 19:52:46.661153+00:00,140.013069,351.633757,8.203125,6,47.648376,-123.636385,8.203125,3,1049251,1.326894,OFR,P,1.293912e+09,2011-01-01 19:52:46.661153+00:00
2,19,4,2011-01-01 21:15:56.139179+00:00,181.291823,464.635512,19.140625,10,48.653486,-123.039309,19.140625,4,1895188,0.487547,LRIV,S,1.293917e+09,2011-01-01 21:15:56.139179+00:00
3,29,5,2011-01-02 02:10:45.376984+00:00,-0.557821,518.990787,49.609375,17,49.168544,-125.507649,49.609375,5,1121863,0.554185,BTB,P,1.293934e+09,2011-01-02 02:10:45.376984+00:00
5,46,6,2011-01-02 04:23:38.382767+00:00,234.842638,457.483502,0.390625,7,48.571463,-122.317539,0.390625,6,1961027,-0.158275,LZB,S,1.293942e+09,2011-01-02 04:23:38.382767+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41625,68797,8958,2015-12-30 10:12:04.084015+00:00,224.801860,458.913904,9.765625,8,48.587986,-122.452629,9.765625,8958,1475194,-0.269381,CLRS,S,1.451470e+09,2015-12-30 10:12:04.084015+00:00
41626,68805,8959,2015-12-30 14:04:12.055910+00:00,126.625365,540.446816,5.859375,6,49.348330,-123.757253,5.859375,8959,374144,-0.009508,CLRS,P,1.451484e+09,2015-12-30 14:04:12.055910+00:00
41627,13816,2192,2015-12-30 17:21:56.886771+00:00,76.421476,-173.323761,19.140625,7,42.936197,-124.563756,19.140625,2192,182969,-0.059408,DBO,P,1.451496e+09,2015-12-30 17:21:56.886771+00:00
41628,68811,8961,2015-12-31 01:10:15.705291+00:00,96.503032,458.913904,38.671875,18,48.620884,-124.191036,38.671875,8961,595258,-0.121397,CLRS,P,1.451524e+09,2015-12-31 01:10:15.705291+00:00


In [3]:
merged_df

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
0,0,0,2011-01-01 04:58:03.504768+00:00,219.223650,447.470689,0.390625,6,48.487194,-122.534153,0.390625,0,1940523,0.030308,D03D,S,1.293858e+09,2011-01-01 04:58:03.504768+00:00
1,13,3,2011-01-01 19:52:46.661153+00:00,140.013069,351.633757,8.203125,6,47.648376,-123.636385,8.203125,3,1049251,1.326894,OFR,P,1.293912e+09,2011-01-01 19:52:46.661153+00:00
2,19,4,2011-01-01 21:15:56.139179+00:00,181.291823,464.635512,19.140625,10,48.653486,-123.039309,19.140625,4,1895188,0.487547,LRIV,S,1.293917e+09,2011-01-01 21:15:56.139179+00:00
3,29,5,2011-01-02 02:10:45.376984+00:00,-0.557821,518.990787,49.609375,17,49.168544,-125.507649,49.609375,5,1121863,0.554185,BTB,P,1.293934e+09,2011-01-02 02:10:45.376984+00:00
4,0,0,2011-01-02 02:10:45.481987+00:00,19.523735,523.281993,49.609375,10,49.206819,-125.232068,49.609375,0,358086,0.215366,W01,P,1.293934e+09,2011-01-02 02:10:45.481987+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41625,68797,8958,2015-12-30 10:12:04.084015+00:00,224.801860,458.913904,9.765625,8,48.587986,-122.452629,9.765625,8958,1475194,-0.269381,CLRS,S,1.451470e+09,2015-12-30 10:12:04.084015+00:00
41626,68805,8959,2015-12-30 14:04:12.055910+00:00,126.625365,540.446816,5.859375,6,49.348330,-123.757253,5.859375,8959,374144,-0.009508,CLRS,P,1.451484e+09,2015-12-30 14:04:12.055910+00:00
41627,13816,2192,2015-12-30 17:21:56.886771+00:00,76.421476,-173.323761,19.140625,7,42.936197,-124.563756,19.140625,2192,182969,-0.059408,DBO,P,1.451496e+09,2015-12-30 17:21:56.886771+00:00
41628,68811,8961,2015-12-31 01:10:15.705291+00:00,96.503032,458.913904,38.671875,18,48.620884,-124.191036,38.671875,8961,595258,-0.121397,CLRS,P,1.451524e+09,2015-12-31 01:10:15.705291+00:00


In [15]:
# import pandas as pd
# from obspy.geodetics import locations2degrees, degrees2kilometers

# # Load all event data into a list of DataFrames
# event_files = [
#     '../data/datasets_nwa_shelf_trench/new_events.csv',
#     '../data/datasets_nwa_shore/new_events.csv',
#     '../data/datasets_or_shelf_trench/new_events.csv',
#     '../data/datasets_or_shore/new_events.csv',
#     '../data/datasets_pnsn_jdf/new_events.csv',
#     '../data/datasets_pnsn_nor/new_events.csv',
#     '../data/datasets_pnsn_sor/new_events.csv',
#     '../data/datasets_pnsn_wa/new_events.csv',
#     '../data/datasets_swa_shelf_trench/new_events.csv',
#     '../data/datasets_swa_shore/new_events.csv'
# ]

# # Read all event files into DataFrames
# dfs = [pd.read_csv(file, index_col=0) for file in event_files]

# # Concatenate all DataFrames into a single DataFrame
# merged_df = pd.concat(dfs)

# # Sort the DataFrame by 'datetime'
# merged_df['datetime'] = pd.to_datetime(merged_df['datetime'])
# merged_df = merged_df.sort_values(by='datetime').reset_index(drop=True)

# # Define thresholds
# time_threshold = pd.Timedelta(seconds=10)
# dist_threshold = 25  # in kilometers

# # Function to calculate distance between two events
# def calculate_distance(lat1, lon1, lat2, lon2):
#     degrees = locations2degrees(lat1, lon1, lat2, lon2)
#     return degrees2kilometers(degrees)

# # Iterate through the DataFrame and remove events within the thresholds
# filtered_events = []
# print('1',filtered_events)
# for i, event in merged_df.iterrows():
#     add_event = True
#         print('2',filtered_events)

#     for last_event in filtered_events:
#         print('3',filtered_events)
#         print(event['idx'],last_event['idx'])
#         time_diff = event['datetime'] - last_event['datetime']
#         distance = calculate_distance(event['latitude'], event['longitude'], last_event['latitude'], last_event['longitude'])
        
#         if time_diff <= time_threshold and distance <= dist_threshold:
#             add_event = False
#             break
    
#     if add_event:
#         filtered_events.append(event)

# # Convert the filtered events back to a DataFrame
# filtered_df = pd.DataFrame(filtered_events)

# # Save the merged and filtered catalog to a new CSV file
# filtered_df.to_csv('../data/merged_filtered_events.csv', index=False)

IndentationError: unexpected indent (676045518.py, line 42)

In [ ]:
# # Load all event data into a list of DataFrames
# event_files = [
#     '../data/datasets_nwa_shelf_trench/new_events.csv',
#     '../data/datasets_nwa_shore/new_events.csv',
#     '../data/datasets_or_shelf_trench/new_events.csv',
#     '../data/datasets_or_shore/new_events.csv',
#     '../data/datasets_pnsn_jdf/new_events.csv',
#     '../data/datasets_pnsn_nor/new_events.csv',
#     '../data/datasets_pnsn_sor/new_events.csv',
#     '../data/datasets_pnsn_wa/new_events.csv',
#     '../data/datasets_swa_shelf_trench/new_events.csv',
#     '../data/datasets_swa_shore/new_events.csv'
# ]

# # Read all event files into DataFrames
# dfs = [pd.read_csv(file, index_col=0) for file in event_files]

# # Concatenate all DataFrames into a single DataFrame
# merged_df = pd.concat(dfs)

# # Sort the DataFrame by 'datetime'
# merged_df['datetime'] = pd.to_datetime(merged_df['datetime'])
# merged_df = merged_df.sort_values(by='datetime').reset_index(drop=True)

# # Define thresholds
# time_threshold = pd.Timedelta(seconds=10)
# dist_threshold = 25  # in kilometers

# # Function to calculate distance between two events
# def calculate_distance(lat1, lon1, lat2, lon2):
#     degrees = locations2degrees(lat1, lon1, lat2, lon2)
#     return degrees2kilometers(degrees)

# # filterd_events = []
# # for i, event1 in merged_df.iterrows():
# #     for j, event2 in merged_df.iterrows():
# #         print('i:',i,'j:',j,"event1['idx']",event1['idx'], "event2['idx']",event2['idx'])
# #         if i == j:
# #             continue
# #         time_diff = event2['datetime'] - event1['datetime']
# #         distance = calculate_distance(event2['latitude'], event2['longitude'], event2['latitude'], event1['longitude'])
        
# #         if time_diff > time_threshold or distance > dist_threshold:
# #             print(f"matched: {i}, {j}")
# #             merged_df.drop(j, inplace=True)

# # filterd_events = []
# # for i in range(len(merged_df)):
# #     for j in range(len(merged_df)):
# #         if i == j:
# #             continue
# #         time_diff = event2['datetime'] - event1['datetime']
# #         distance = calculate_distance(event['latitude'], event['longitude'], merged_df.loc[j,'latitude'], merged_df.loc[j,'longitude'])
        
# #         if time_diff > time_threshold or distance > dist_threshold:
# #             print(f"matched: {i}, {j}")
# #             merged_df.drop(j, inplace=True)


# # Initialize a list to collect indices of rows to be dropped


# for i in tqdm(range(len(merged_df)),total=len(merged_df)):
#     if i in rows_to_drop:
#             continue
#     event1 = merged_df.loc[i]
#     for j in range(i+1,len(merged_df)):
#         if j in rows_to_drop:
#             continue
#         event2 = merged_df.loc[j]
#         time_diff = event2['datetime'] - event1['datetime']
#         distance = calculate_distance(event2['latitude'], event2['longitude'], event1['latitude'], event1['longitude'])
        
#         if time_diff <= time_threshold and distance <= dist_threshold:
#             rows_to_drop.append(j)

# # Drop the collected rows
# merged_df.drop(rows_to_drop, inplace=True)

# # Reset the index to ensure it is sequential
# merged_df.reset_index(drop=True, inplace=True)



# # rows_to_drop = []

# # for i in range(len(merged_df)):
# #     event1 = merged_df.loc[i]
# #     for j in range(i+1,len(merged_df)):
# #         if i==j or j in rows_to_drop:
# #             continue
# #         event2 = merged_df.loc[j]
# #         time_diff = event2['datetime'] - event1['datetime']
# #         distance = calculate_distance(event2['latitude'], event2['longitude'], event1['latitude'], event1['longitude'])
        
# #         if time_diff <= time_threshold and distance <= dist_threshold:
# #             rows_to_drop.append(j)

# # # Drop the collected rows
# # merged_df.drop(rows_to_drop, inplace=True)

# # # Reset the index to ensure it is sequential
# # merged_df.reset_index(drop=True, inplace=True)

  3%|▎         | 1234/41630 [1:24:25<35:48:38,  3.19s/it]

In [ ]:
merged_df.to_csv('../data/merged_filtered_events.csv', index=False)